In [5]:
import cv2 as cv

face_cascade = cv.CascadeClassifier('./Ressources/haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('./Ressources/haarcascade_eye.xml')

img = cv.imread('../Image/img_label_3.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Dimension de l'image capturée pour avoir des données uniforme.
desired_width = 24
desired_height = 24

scale_factor = 1.3 # Un facteur plus petit augmente la chance de détecter des visages plus petits, mais peut aussi augmenter les faux positifs.
min_neighbors = 5 # Augmenter cette valeur réduit les faux positifs. Cela définit le nombre de voisins qu'un rectangle doit avoir pour être retenu.

nb_label = 30;

In [8]:
def vectorize_both_eyes(img, show_results=False):
    # Convertir l'image en niveaux de gris
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Détecter les yeux dans l'image
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
    
    # Initialiser les vecteurs des yeux comme None
    vecteur_oeil_droit, vecteur_oeil_gauche = None, None
    
    # Trier les yeux détectés par leur coordonnée x (le plus à gauche d'abord)
    sorted_eyes = sorted(eyes, key=lambda x: x[0])
    
    # Assigner l'œil gauche et droit en fonction de leur position x
    for i, eye in enumerate(sorted_eyes):
        x, y, w, h = eye
        eye_region = gray[y:y+h, x:x+w]
        vecteur = eye_region.flatten()
        if i == 0:  # Premier œil (le plus à gauche)
            vecteur_oeil_gauche = vecteur
        elif i == 1:  # Deuxième œil (le plus à droite)
            vecteur_oeil_droit = vecteur
        if show_results:
            cv.imshow(f'Œil {i+1}', eye_region)
    
    # Afficher les résultats si demandé
    if show_results and vecteur_oeil_gauche is not None and vecteur_oeil_droit is not None:
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    return vecteur_oeil_droit, vecteur_oeil_gauche


In [9]:
# Étape 1 : Lire le Fichier JSON
import json
import numpy as np
import cv2 as cv

# Charger le fichier JSON
with open('../Data/sorted_by_label.json', 'r') as file:
    data = json.load(file)

# Utiliser seulement les nb_label premiers labels
selected_labels = list(data['data'].keys())[:nb_label]

# Étape 2 : Calculer les Vecteurs pour Chaque Œil de Chaque Image
vectors_by_class = {label: {'right_eye': [], 'left_eye': []} for label in selected_labels}

for label in selected_labels:
    for image_name in data['data'][label]:
        img_path = f'../Image/img_align_celeba/img_align_celeba/{image_name}'
        img = cv.imread(img_path)
        if img is not None:
            # Supposons que vectoriser_oeil_v2 retourne un tuple (vecteur_oeil_droit, vecteur_oeil_gauche)
            vecteur_oeil_droit, vecteur_oeil_gauche = vectorize_both_eyes(img, show_results=False)
            if vecteur_oeil_droit is not None:
                vectors_by_class[label]['right_eye'].append(vecteur_oeil_droit)
            if vecteur_oeil_gauche is not None:
                vectors_by_class[label]['left_eye'].append(vecteur_oeil_gauche)

# Étape 3 : Calculer le Vecteur Moyen pour l'Œil Droit et l'Œil Gauche de Chaque Label
mean_vectors = {label: {'right_eye': None, 'left_eye': None} for label in selected_labels}

for label, eyes_vectors in vectors_by_class.items():
    for eye, vectors in eyes_vectors.items():
        if vectors:
            mean_vectors[label][eye] = np.mean(vectors, axis=0).tolist()

# Étape 4 : Enregistrer en Format JSON
with open('../Data/mean_vectors_by_eye.json', 'w') as json_file:
    json.dump(mean_vectors, json_file, indent=4)
